In [14]:
import pickle
import random
import numpy as np
import os
import os.path as osp
import plotly.express as px

In [3]:
with open('./NTU/X.pkl', 'rb') as f:
    X = pickle.load(f)

to_del = []
for file in X:
    if type(X[file]) == list:
        to_del.append(file)
print('to delete', len(to_del))
for file in to_del:
    del X[file]

to delete 28814


In [5]:
d = []
for key in X:
    d = X[key][0]
    break
d.shape

(75,)

In [16]:
reshaped_data = d.reshape(-1, 3)
x = reshaped_data[:, 0]
y = reshaped_data[:, 1]
z = reshaped_data[:, 2]

import pandas as pd
df = pd.DataFrame({'x': x, 'y': y, 'z': z})

# Create the interactive 3D scatter plot
fig = px.scatter_3d(df, x='x', y='y', z='z', color=np.linspace(1, 25, len(x)),
                     color_continuous_scale='Rainbow', title='Interactive 3D Scatter Plot')

fig.update_traces(marker=dict(size=5))

# Show the plot
fig.show()